<a href="https://colab.research.google.com/github/bhavanabalraj/Federated-Learning/blob/master/DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install torch

In [8]:
import torch

num_entries = 5000

db = torch.rand(num_entries) > 0.5
db.shape

db[0:5]

tensor([ True, False,  True, False, False])

In [9]:
dbList = []

for i in range(num_entries):
  y = db.clone().detach()
  y = torch.cat([y[0:i],y[i+1:]])
  dbList.append(y)

dbList[0].shape

torch.Size([4999])

In [0]:
def get_parallel_db(db, index):
  return torch.cat([db[0:index],db[index+1:]])

def get_parallel_dbs(db):

  parallel_dbs = list()

  for i in range(len(db)):
    pdb = get_parallel_db(db,i)
    parallel_dbs.append(pdb)

  return parallel_dbs

def create_db_and_parallels(num_entries):
  
  db = torch.rand(num_entries) > 0.5
  return db, get_parallel_dbs(db)


In [11]:
db, pdbs = create_db_and_parallels(20)
pdbs

[tensor([False, False,  True, False,  True, False, False, False, False, False,
          True, False,  True, False,  True, False,  True,  True, False]),
 tensor([ True, False,  True, False,  True, False, False, False, False, False,
          True, False,  True, False,  True, False,  True,  True, False]),
 tensor([ True, False,  True, False,  True, False, False, False, False, False,
          True, False,  True, False,  True, False,  True,  True, False]),
 tensor([ True, False, False, False,  True, False, False, False, False, False,
          True, False,  True, False,  True, False,  True,  True, False]),
 tensor([ True, False, False,  True,  True, False, False, False, False, False,
          True, False,  True, False,  True, False,  True,  True, False]),
 tensor([ True, False, False,  True, False, False, False, False, False, False,
          True, False,  True, False,  True, False,  True,  True, False]),
 tensor([ True, False, False,  True, False,  True, False, False, False, False,
   

In [12]:
def query(db, threshold):
  return (db.sum() > threshold).float()

def sensitivity(query, num_entries):
  
  db, pdbs = create_db_and_parallels(num_entries)

  full_db_result = query(db, 5)

  sensitivity = 0

  for pdb in pdbs:
    pdb_result = query(pdb, 5)
    distance = torch.abs(full_db_result - pdb_result)

    if distance > sensitivity:
      sensitivity = distance

  return sensitivity

sensitivityList = list()

for i in range(10):
  s = sensitivity(query, 10)
  sensitivityList.append(s)

sensitivityList

[0, 0, tensor(1.), tensor(1.), 0, 0, 0, 0, 0, 0]

In [14]:
def query(db):
  return db.sum()

def differencing(db, pdb):
  full_db_result = query(db)
  pdb_result = query(pdb)
  return full_db_result == pdb_result

db, pdbs = create_db_and_parallels(100)
pdb = get_parallel_db(db, 5)
differencing(db, pdb)



tensor(True)

In [18]:
import random

def query(db):
  return db.float().mean()

def randomDb(db, weight):

#  for i in range(len(db)):
#    chance1 = random.randint(0,1)
#    if chance1 == 0:
#        db[i] = random.randint(0,1)
#  return db

  #weights = torch.tensor([weight, 1-weight], dtype=torch.float)
  #print(torch.multinomial(weights, len(db), replacement=True))
  first_coin_flip = (torch.rand(len(db)) > 0.5).float()
  #first_coin_flip = torch.multinomial(weights, len(db), replacement=True).float()
  second_coin_flip  = (torch.rand(len(db)) > 0.5).float()

  augmented_db = (db.float() * first_coin_flip) + (1 - first_coin_flip) * second_coin_flip
  return augmented_db

def testRandomNoise(n_entries):
  
  db, pdbs = create_db_and_parallels(n_entries)
  #print(db)
  db_result = query(db)

  randomized_db = randomDb(db,0.50)
  #print(randomized_db)
  randomDb_result = query(randomized_db) * 2 - 0.5

  return db_result, randomDb_result

for i in [10,100,1000,10000]:
  actual, predicted = testRandomNoise(i)
  print(actual)
  print(predicted)


tensor(0.6000)
tensor(0.3000)
tensor(0.6100)
tensor(0.7200)
tensor(0.5160)
tensor(0.5040)
tensor(0.5026)
tensor(0.5112)


In [23]:
import random

def query(db):
  return db.float().mean()

def randomDb(db, weight):

#  for i in range(len(db)):
#    chance1 = random.randint(0,1)
#    if chance1 == 0:
#        db[i] = random.randint(0,1)
#  return db

  #weights = torch.tensor([weight, 1-weight], dtype=torch.float)
  #print(torch.multinomial(weights, len(db), replacement=True))
  first_coin_flip = (torch.rand(len(db)) > weight).float()
  #first_coin_flip = torch.multinomial(weights, len(db), replacement=True).float()
  second_coin_flip  = (torch.rand(len(db)) > 0.5).float()

  augmented_db = (db.float() * first_coin_flip) + (1 - first_coin_flip) * second_coin_flip
  return augmented_db

def testRandomNoise(n_entries):
  
  db, pdbs = create_db_and_parallels(n_entries)
  #print(db)
  db_result = query(db)

  noise = 0.5
  randomized_db = randomDb(db, noise)
  #print(randomized_db)
  randomDb_result = ((query(randomized_db) / noise) - 0.5) * (noise / (1 - noise))

  return db_result, randomDb_result

for i in [10,100,1000,10000]:
  actual, predicted = testRandomNoise(i)
  print(actual)
  print(predicted)


tensor(0.6000)
tensor(0.5000)
tensor(0.4600)
tensor(0.4000)
tensor(0.5170)
tensor(0.5540)
tensor(0.4965)
tensor(0.5064)


In [0]:
def query(db):
  return db.sum() 

db, pdbs = create_db_and_parallels(num_entries)

loc, scale = 0., 1.
s = np.random.laplace(loc, scale, 1000)
db_result = query(db) + s

'''
epsilon = 5
b = sensitivity(query, 100) / 5

np.random.laplace()
'''